# Welcome!

Welcome to this tutorial on creating a Gradio demo from scratch for a bot that composes guitar pieces. By the end of this tutorial, you should have an online demo similar to [this one](https://huggingface.co/spaces/juancopi81/mutopia-guitar-composer). To follow this tutorial, you should be familiar with Python and [Transformers](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)) (the machine learning model).

There are different ways to generate music using machine learning. One of those is training the model with symbolic music, that is: "Music stored in a notation-based format (e.g., MIDI), which contains explicit information about note onsets and pitch on individual tracks (for different instruments), but in contrast to Digital Audio, no sound." (https://www.igi-global.com/dictionary/music-information-retrieval/28927). 

You could also model music directly as raw audio as [OpenAI's Jukebox](https://openai.com/blog/jukebox/). There are advantages and disadvantages to each approach. For this tutorial, you'd use the first approach and generate music symbolically. 

This tutorial consists of a collection of five notebooks that you should follow in order. The process that you'll observe is the following:

1.   In the first notebook (this one), you'll collect the training dataset as MIDI files. In this tutorial, you will get the data from the Mutopia Project. Feel free to use other guitar MIDI files you might have (the more files, the better).
2.   In the second notebook, you'll convert the MIDI files into text, which is the core idea behind what you are doing: By having your guitar pieces as text, you could use the well-established NLP models for casual language modeling. To encode the MIDI files into text tokens, you'll use the [implementation](https://github.com/AI-Guru/MMM-JSB) of [Dr. Tristan Beheren](https://www.linkedin.com/in/dr-tristan-behrens-734967a2/) of the paper ["MMM: Exploring Conditional Multi-Track Music Generation with the Transformer"](https://arxiv.org/abs/2008.06048).
3.   Then you'll use [Hugging Face](https://huggingface.co/) to create a new tokenizer for your dataset (notebook 3) and train a TensorFlow version of a GPT-2 model (notebook 4) for text generation.
4.   Finally, in the fifth and final notebook of this tutorial, you take the model you trained and share it with the world building a Gradio demo. With this demo, the results of your model will change from text tokens to sound that users can listen to and a piano roll to visualize the results. You'll use Gradio Blocks to have more control over your demo.

Very exciting! Let's start with the first step of the process: 

# 1. Collecting the data

As mentioned above, in this tutorial, you'll collect the data from [the Mutopia Project](https://www.mutopiaproject.org/). The Mutopia Project is a collection of "*2124 pieces of music – free to download, modify, print, copy, distribute, perform, and record – all in the Public Domain or under Creative Commons licenses, in PDF, MIDI, and editable LilyPond file formats*". This project has the precise type of data you need to train your model!

Please Go and check the website before moving on 🔍. Take special attention to the [guitar section](https://www.mutopiaproject.org/cgibin/make-table.cgi?Instrument=Guitar) of the project.

On the project website, you can notice that you can filter by instrument. You'll use only solo guitar pieces for this tutorial, but you can adapt the code to any other instrument you want. 

You'll use the code explained in [this link](https://www.datacamp.com/tutorial/using-tensorflow-to-compose-music#appendix) for scraping the MIDI files from the website. Notice the use of `BeautifulSoup` to parse the HTML return by `urlopen`. After parsing the HTML, you'll identify the links that are MIDI files and save them to the folder specified in `save_dir`. Finally, as explained in the docs for this code, you'll use `time.sleep` to avoid putting a strain on the website's resources.

You should run this code on your local computer and save the MIDI files in a folder that you can use for the second notebook of this tutorial.

In [ ]:
# Taken from
# https://www.datacamp.com/tutorial/using-tensorflow-to-compose-music#appendix 

from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import time

# Define save directory
save_dir = "./custom_midi_dataset/"

# Define URL components
url0 = "https://www.mutopiaproject.org/cgibin/make-table.cgi?startat="
url1 = "&searchingfor=&Composer=&Instrument=Guitar&Style=&collection=&id=&solo=&recent=&timelength=&timeunit=&lilyversion=&preview="

# Set initial values
songNumber = 0
linkCount = 10

# Locate and download each MIDI file
while linkCount > 0:
    url = url0 + str(songNumber) + url1
    html = urlopen(url)
    soup = BeautifulSoup(html.read())
    links = soup.find_all("a")
    linkCount = 0
    for link in links:
        href = link["href"]
        if href.find(".mid") >= 0:
            title = href.split("/")[-1]
            print(title)
            linkCount += 1
            urlretrieve(href, save_dir+title)
    songNumber += 10
    time.sleep(10.0)

If everything went fine, you should now have 395 (as of September 29, 2022) MIDI files in the folder `./custom_midi_dataset/.` Please review these files: Feel free to hear them and learn more about the composer. To explore these MIDI files, you can use free software, like [MuseScore](https://musescore.org/). 

While doing this, please remove all the files that are not guitar solo pieces (there are some duets and other formats).

Great job! You now have your raw data for training your model. You'll take the MIDI files in the following notebook and convert them into text tokens. Exciting right?